# Data Augmentation Translation

## Imports

In [1]:
import pandas as pd
import os
import uuid
import requests
from tqdm import tqdm
import time
from dotenv import load_dotenv

In [2]:
tqdm.pandas()

## Azure AI Translation

In [3]:
load_dotenv()

True

In [4]:
key = os.getenv("API_AZURE_KEY")
location = os.getenv("API_AZURE_LOCATION")
endpoint = "https://api.cognitive.microsofttranslator.com"
path = "/translate"

In [5]:
constructed_url = endpoint + path

headers = {
    'Ocp-Apim-Subscription-Key': key, 
    'Ocp-Apim-Subscription-Region': location, 
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

In [6]:
def translate_text(text, from_lang, to_lang, max_retries=5):
    if not text:
        return None

    params = {
        'api-version': '3.0',
        'from': from_lang,
        'to': [to_lang]
    }
    body = [{'text': text}]
    retries = 0

    while retries <= max_retries:
        try:
            response = requests.post(constructed_url, params=params, headers=headers, json=body)
            response.raise_for_status()  
            translated_text = response.json()[0]['translations'][0]['text']
            return translated_text
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429: 
                retries += 1
                wait_time = 2 ** retries 
                time.sleep(wait_time)
            else:
                print(f"HTTP Error: {e}")
                break
        except Exception as e:
            print(f"Error translating text: {e}")
            break
            
    return None

In [18]:
def compare_translations(df, index):
    
    label = df.loc[index, 'rating']
    original_text = df.loc[index, 'reviewText'] 
    german_text = df.loc[index, 'reviewText_german']
    german_french_text = df.loc[index, 'reviewText_german_french']
    final_text = df.loc[index, 'reviewText_german_french_english'] 
    
    print(f"Index: {index}")
    print(f"****")
    print(f"Label: {label}")
    print(f"****")
    print(f"Original Text: {original_text}")
    print(f"****")
    print(f"German Translation: {german_text}")
    print(f"****")
    print(f"German-French Translation: {german_french_text}")
    print(f"****")
    print(f"Back-Translated Text: {final_text}")
    print("")

## Translate data

In [9]:
df = pd.read_csv("./data/combined_reviews.csv")

In [10]:
df.head(2)

,rating,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,reviewToken
0,5.0,2017-01-16,ASWLL1VJA7WOG,Great product... just what I wanted. Works gr...,Five Stars,1484524800,All_Beauty,"['great', 'product', 'want', 'works', 'great',..."
1,5.0,2008-12-08,A265K3A7V83112,"After seeing the popularity of this shoe, I de...",What can i say? chucks rock,1228694400,Clothing_Shoes_and_Jewelry,"['see', 'popularity', 'shoe', 'decide', 'test'..."


In [12]:
df['reviewText_german'] = df.progress_apply(
    lambda row: translate_text(row['reviewText'], from_lang='en', to_lang='de'), axis=1)

  1%|          | 69/12000 [00:14<37:15,  5.34it/s] 

Error translating text: Out of range float values are not JSON compliant


  1%|          | 118/12000 [00:23<20:58,  9.44it/s]

Error translating text: Out of range float values are not JSON compliant


  6%|▌         | 734/12000 [02:15<29:57,  6.27it/s]  

...

 96%|█████████▌| 11517/12000 [30:56<00:50,  9.58it/s]

Error translating text: Out of range float values are not JSON compliant


100%|█████████▉| 11964/12000 [32:06<00:03,  9.95it/s]

Error translating text: Out of range float values are not JSON compliant


100%|██████████| 12000/12000 [32:12<00:00,  6.21it/s]


In [13]:
df['reviewText_german_french'] = df.progress_apply(
    lambda row: translate_text(row['reviewText_german'], from_lang='de', to_lang='fr'), axis=1)

100%|██████████| 12000/12000 [42:50<00:00,  4.67it/s] 


In [14]:
df['reviewText_german_french_english'] = df.progress_apply(
    lambda row: translate_text(row['reviewText_german_french'], from_lang='fr', to_lang='en'), axis=1)

100%|██████████| 12000/12000 [31:02<00:00,  6.44it/s] 


## Check translations

In [19]:
compare_translations(df,2)

Index: 2
****
Label: 5.0
****
Original Text: I was nervousness about the scent because IVe never tried this, but I love Paul Mitchell so I decided to try it.  It smells great!!!  This is a medium hold spray, so I'll use it when my hair is down (I use a stronger hold when I put my hair up).  Shipping speed was good to Alaska too.
****
German Translation: Ich war nervös wegen des Duftes, weil ich ihn noch nie probiert habe, aber ich liebe Paul Mitchell, also beschloss ich, ihn auszuprobieren.  Es riecht gut!!  Dies ist ein Spray mit mittlerem Halt, daher verwende ich es, wenn mein Haar offen ist (ich verwende einen stärkeren Halt, wenn ich mein Haar hochstecke).  Die Versandgeschwindigkeit nach Alaska war auch gut.
****
German-French Translation: J’étais nerveuse à propos de l’odeur parce que je ne l’avais jamais essayé auparavant, mais j’adore Paul Mitchell, alors j’ai décidé de l’essayer.  Ça sent bon !!  Il s’agit d’un spray à tenue moyenne, donc je l’utilise quand mes cheveux sont ou

## Save new df

In [42]:
output_path = './data/reviews_da_translation_wide.csv'
df.to_csv(output_path, index=False)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   rating                            12000 non-null  float64
 1   reviewTime                        12000 non-null  object 
 2   reviewerID                        12000 non-null  object 
 3   reviewText                        11975 non-null  object 
 4   summary                           11991 non-null  object 
 5   unixReviewTime                    12000 non-null  int64  
 6   category                          12000 non-null  object 
 7   reviewToken                       12000 non-null  object 
 8   reviewText_german                 11964 non-null  object 
 9   reviewText_german_french          11964 non-null  object 
 10  reviewText_german_french_english  11964 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1.0+ MB


In [38]:
long_df = pd.melt(
    df,
    id_vars=['rating'],
    value_vars=['reviewText', 'reviewText_german_french_english'],
    var_name='text_type',
    value_name='reviewText_long' 
)

In [39]:
long_df.rename(columns={'reviewText_long': 'reviewText'}, inplace=True)

In [41]:
output_path = './data/reviews_da_translation_long.csv'
long_df.to_csv(output_path, index=False)